In [2]:
import pandas as pd
import numpy as np
import datetime as dt
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
import seaborn as sn
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import funcoes
import itertools
from sklearn.metrics import accuracy_score

sns.set_style('darkgrid')
%matplotlib inline

In [3]:
df = pd.read_csv('conhecimento.csv')
df['dia'] = df.id_cena.str[-3:].astype(int)

id_cenas = [
    # 'LS82260702016078', 
    'LS82260702017256', # 226_070
    # 'LS82210702016059', 
    'LS82210702017269', # 221_070
    # 'LS52210672002068', 
    'LS82210672017285', # 221_067
    # 'LS82210742016059', 
    'LS82210742018128', # 221_074
]
col_sum = ['previsao', 'verdade', 'porcentagem', 'id_cena', 'caso']

In [4]:
def gera_teste_modelo_caso2(id_cena):
    
    teste_original = df[df.id_cena==id_cena].copy().reset_index(drop=True)
    teste = teste_original.copy()
    teste['verifica'] = 0

    # Modelo
    modelo_inteiro = df[~(df.id_cena==id_cena)].copy()

    modelo_inteiro_1 = modelo_inteiro[modelo_inteiro.verifica==1].copy()
    modelo_inteiro_3_full = modelo_inteiro[modelo_inteiro.verifica==3].copy()

    id3 = np.random.choice(modelo_inteiro_3_full.id, len(modelo_inteiro_1), replace=False) 
    modelo_inteiro_3 = modelo_inteiro_3_full[modelo_inteiro_3_full.id.isin(id3)]

    modelo = pd.concat([modelo_inteiro_1, modelo_inteiro_3]).reset_index(drop=True)

    return teste, teste_original, modelo
def resultado(teste, teste_original, pred, verdade,verifica=1):
    r = pd.merge(teste[['id', pred]], teste_original[['id', verdade]], on='id')
    r['v'] = r[pred].astype(str) + r[verdade].astype(str)
    
    try:
        x = len(r[(r[verdade]==verifica) & (r[pred] == verifica)])
        v = len(r[r[verdade]==verifica])
        p = int((x*100)/v)
    except:
        return [0, 0, 0]
    return [x, v, p]

In [15]:
def class_rf(dado, modelo, colunas):
    classificador = RandomForestClassifier(n_jobs=8, n_estimators=100, max_features=len(melhores_colunas), random_state=0, max_depth=3, min_samples_split=5)
    classificador.fit(modelo[colunas], modelo.verifica)
    return classificador.predict(dado[colunas])

def class_nn(dado, modelo, colunas):
    classificador = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam', alpha=0.00010, max_iter=200)
    classificador.fit(modelo[colunas], modelo.verifica)
    return classificador.predict(dado[colunas])

In [5]:
melhores_colunas = ['ndvi', 'nbrl', 'dif_ndvi', 'dif_dnbrl', 'medianb2', 'medianb3', 'medianb4', 'medianb5', 'medianb6', 'medianb7']

In [16]:
col_sum = ['previsao', 'verdade', 'porcentagem', 'id_cena', 'caso']
sumario = pd.DataFrame([], columns=col_sum)
for id_cena in id_cenas:
    teste, teste_original, modelo = gera_teste_modelo_caso2(id_cena)
    
    nome_classe = 'rf'
    teste[nome_classe] = class_rf(teste, modelo, melhores_colunas)
    y_pred = teste.sort_values('id').reset_index(drop=True)
    y_true = teste_original.sort_values('id').reset_index(drop=True)
    r = resultado(y_pred, y_true, nome_classe, 'verifica')
    r.extend([id_cena, nome_classe])
    print(r)
    sumario = sumario.append(dict(zip(col_sum, r)), ignore_index=True)
    
    nome_classe = 'nn'
    teste[nome_classe] = class_nn(teste, modelo, melhores_colunas)
    y_pred = teste.sort_values('id').reset_index(drop=True)
    y_true = teste_original.sort_values('id').reset_index(drop=True)
    r = resultado(y_pred, y_true, nome_classe, 'verifica')
    r.extend([id_cena, nome_classe])
    print(r)
    sumario = sumario.append(dict(zip(col_sum, r)), ignore_index=True)
    
    teste['avaliacao'] = (teste["rf_1"] + teste["nn_1"]) / 2
    teste['avaliacao'] = teste['avaliacao'].apply(lambda x: round(x, 4))
    
    teste['classificacao'] = teste.avaliacao.apply(lambda x: 1 if x > 0.9 else 3)
    y_pred = teste.sort_values('id').reset_index(drop=True)
    y_true = teste_original.sort_values('id').reset_index(drop=True)
    r = resultado(y_pred, y_true, 'classificacao', 'verifica')
    r.extend([id_cena, 'class'])
    print(r)
    sumario = sumario.append(dict(zip(col_sum, r)), ignore_index=True)
    
#     teste['tmp'] = teste.nn.astype(str)+teste.rf.astype(str)
#     teste['tmp'] = teste['tmp'].apply(lambda x: 1 if x == '11' else 3)
#     teste['tmp'].value_counts()
    
#     y_pred = teste.sort_values('id').reset_index(drop=True)
#     y_true = teste_original.sort_values('id').reset_index(drop=True)
    
#     r = resultado(y_pred, y_true, 'tmp', 'verifica')
#     r.extend([id_cena, 'tmp'])
#     print(r)

[805, 805, 100, 'LS82260702017256', 'rf']
[548, 599, 91, 'LS82210702017269', 'rf']
[1088, 1102, 98, 'LS82210672017285', 'rf']
[917, 1072, 85, 'LS82210742018128', 'rf']


In [9]:
sumario

,previsao,verdade,porcentagem,id_cena,caso
0,805,805,100,LS82260702017256,rf_av1
1,805,805,100,LS82260702017256,rf
2,705,805,87,LS82260702017256,nn_av1
3,798,805,99,LS82260702017256,nn
4,741,805,92,LS82260702017256,class
5,523,599,87,LS82210702017269,rf_av1
6,547,599,91,LS82210702017269,rf
7,508,599,84,LS82210702017269,nn_av1
8,532,599,88,LS82210702017269,nn
9,503,599,83,LS82210702017269,class


In [10]:
sumario2=sumario.copy()

In [8]:
dados = [[805, 805, 100, 'LS82260702017256', 'rf_av1'],
[805, 805, 100, 'LS82260702017256', 'rf'],
[705, 805, 87, 'LS82260702017256', 'nn_av1'],
[798, 805, 99, 'LS82260702017256', 'nn'],
[741, 805, 92, 'LS82260702017256', 'class'],
[798, 805, 99, 'LS82260702017256', 'tmp'],
[523, 599, 87, 'LS82210702017269', 'rf_av1'],
[547, 599, 91, 'LS82210702017269', 'rf'],
[508, 599, 84, 'LS82210702017269', 'nn_av1'],
[532, 599, 88, 'LS82210702017269', 'nn'],
[503, 599, 83, 'LS82210702017269', 'class'],
[530, 599, 88, 'LS82210702017269', 'tmp'],
[882, 1102, 80, 'LS82210672017285', 'rf_av1'],
[1088, 1102, 98, 'LS82210672017285', 'rf'],
[990, 1102, 89, 'LS82210672017285', 'nn_av1'],
[1088, 1102, 98, 'LS82210672017285', 'nn'],
[935, 1102, 84, 'LS82210672017285', 'class'],
[1083, 1102, 98, 'LS82210672017285', 'tmp'],
[724, 1072, 67, 'LS82210742018128', 'rf_av1'],
[911, 1072, 84, 'LS82210742018128', 'rf'],
[738, 1072, 68, 'LS82210742018128', 'nn_av1'],
[966, 1072, 90, 'LS82210742018128', 'nn'],
[697, 1072, 65, 'LS82210742018128', 'class'],
[857, 1072, 79, 'LS82210742018128', 'tmp']]

In [11]:
col_sum = ['previsao', 'verdade', 'porcentagem', 'id_cena', 'caso']
sumario = pd.DataFrame([], columns=col_sum)

for r in dados:
    sumario = sumario.append(dict(zip(col_sum, r)), ignore_index=True)

In [12]:
sumario

,previsao,verdade,porcentagem,id_cena,caso
0,805,805,100,LS82260702017256,rf_av1
1,805,805,100,LS82260702017256,rf
2,705,805,87,LS82260702017256,nn_av1
3,798,805,99,LS82260702017256,nn
4,741,805,92,LS82260702017256,class
5,798,805,99,LS82260702017256,tmp
6,523,599,87,LS82210702017269,rf_av1
7,547,599,91,LS82210702017269,rf
8,508,599,84,LS82210702017269,nn_av1
9,532,599,88,LS82210702017269,nn
